# Comapany Background:

Select  Collections,  Inc.,  was  a  start-up  subsidiary  of  a  major credit  card  company

***The company  purchased  distressed  consumer  debt  at  discounted  rates from  such  major  credit  card***
and  then  
used  data-driven  decision-making and dynamic value assessment to optimize the collection processes associated with the purchased accounts

For each purchased account, ***the first decision was whether to resell or attempt to collect***. For accounts the company decided to attempt to collect, a host of tactics was available for use in any sequence and with any frequency. 

Like other collection companies, Select Collections used ***the telephone  and  the  legal  system***  as  its  two  major  collection  tools.  



# Goal:


I want to see if we can ***build a model to predict how much money we will collect from  delinquent  accounts.***  Such  a  model  would  prove  very  useful in  ***deciding which accounts to purchase and how much to pay.***

There are ***3,570*** accounts in a data set that I will call the training set

***We  purchased  these  accounts  and  then  collected  the  amount  shown  in  the  last column, labeled totalpay.*** 
Totalpay is the variable of interest. I want you to ***come up with a way to forecast totalpay.***

I  want  you  to  use  the  model  you  come  up with to predict totalpay for the 3,570 accounts in a separate worksheet of data that I will call the test set. 

I  want  an  Excel  file  from  you  containing a single column of numbers. 
***In cell A1, please put the adjusted R-squared for your model.***
***In cells A2 through A3571, put your predicted values for the accounts in the test data set in ID order.***



# Datadict: 
![alt text](datadict.png "data dict")

# Data description:

![alt text](description.png "data description")

# Colume Comparation to total pay

![alt text](col_comparasion_totalpay.png "colume vs totalpay description")

# Col vs Col description
![alt text](col_to_col_relationship.png "col vs col description")

# Business Understanding


# Understanding the data
#### From data dict :
what col should be care?

#### from data description:
data distrbution is fair ? 
fair in total
rollout is also important ?
not fair

#### from col to total pay part we now only 4 cols are not well distributed

inacscr: The natural log of the ***accessscr***

***accessscr***: The account’s accessibility score, an a priori estimation of how likely it is that the accountholder is reached via phone.
bureauscr: The output of an in-house prediction model based on the likelihood of receiving payments from the accountholder using credit-bureau attributes as predictors 

numcalls: The number of telephone calls made (to date) to the accountholder

numrpcs: The number of “right-party connects” or phone calls in which the collection agent speaks with the accountholder 

#### how about col to col ?

what grid means to me ?


# Data Preparation

From ***Understanding the data parts***, already shown the data related accessibility is important

In [ ]:
## import pacakge
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns


labeled_data_file_name = "Labeled_data.csv"
unlabel_data_file_name = "Unlabeled_Data.csv"
## load the data 
col = ['acctid', 'state', 'zip', 'rollout', 'cobal', 'collscr', 'cs', 'accessscr', 'lnacscr', 'bureauscr', 'eaglemod', 'numcalls', 'numrpcs','totalpay']
label_data_df = pd.read_csv(labeled_data_file_name)
unlabel_data_df = pd.read_csv(unlabel_data_file_name)

# Round1 
feature_col = ['acctid', 'state', 'zip', 'rollout', 'cobal', 'collscr', 'cs', 'accessscr', 'lnacscr', 'bureauscr', 'eaglemod', 'numcalls', 'numrpcs']


## seperate the feature and target
features = label_data_df.drop('totalpay', axis=1)
target = label_data_df.drop(feature_col, axis=1)
features = data.drop(['acctid', 'totalpay'], axis=1)
target = data['totalpay']
## splite the data

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.metrics import mean_squared_error

# Load the data
data = pd.read_csv('Labeled_data.csv')

# Define features and target variable
X = data[['state', 'zip','rollout','cobal','collscr','cs','accessscr','lnacscr','bureauscr','eaglemod','numcalls','numrpcs']]
y = data['totalpay']

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=1337)


260     1128
1805      75
3009    2525
3288    4080
3189     100
Name: totalpay, dtype: int64

# Modeling 

#### Representation

What algorithm we are using ? at least it support R-squared

Linear Regression: Linear regression is a simple and commonly used algorithm that predicts a continuous target variable based on one or more independent variables.

Polynomial Regression: Polynomial regression is a form of regression analysis in which the relationship between the independent variable x and the dependent variable y is modelled as an nth degree polynomial. 

In [ ]:
data = pd.read_csv('Labeled_data.csv')

# Define features and target variable
X = data[['state', 'zip','rollout','cobal','collscr','cs','accessscr','lnacscr','bureauscr','eaglemod','numcalls','numrpcs']]
y = data['totalpay']

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=1337)
X_train_encode = pd.get_dummies(X_train, columns=['state', 'zip', 'rollout', 'collscr'],drop_first=True)


model = RandomForestRegressor(max_depth=6, min_samples_split=15, min_samples_leaf=5, random_state=1337)
model.fit(X_train_encode, y_train)

X_test_encode = pd.get_dummies(X_test, columns=['state', 'zip', 'rollout', 'collscr'],drop_first=True)
X_test_encode = X_test_encode.reindex(columns=X_train_encode.columns, fill_value=0)
predictions = model.predict(X_test_encode)

# Calculate mean squared error
mse = mean_squared_error(y_test, predictions)
print(f"Mean Squared Error on test set: {mse}")

Mean Squared Error on test set: 856513.9636756183


#### Optimization

#### Evaluation